# 使用 MindSpore 复现 MRSA 活性预测

**目标**: 本 Notebook 调用 `mindspore_chem` 包中的模块，来训练、验证并测试一个用于预测 MRSA 活性的图神经网络模型。

## 1. 导入必要的库和模块

In [ ]:
import pandas as pd
import mindspore
from mindspore import context

# 从我们创建的包中导入核心功能
from mindspore_chem.data import split_data
from mindspore_chem.train import run_training, run_testing

# 设置MindSpore环境
context.set_context(mode=context.PYNATIVE_MODE, device_target="CPU")

## 2. 定义实验参数

我们将所有可调参数集中定义在这里，方便修改和管理。

In [ ]:
class Args:
    # --- 数据和路径参数 ---
    # !!! 请务必将 'your_data.csv' 替换为您真实的数据文件名
    data_path = 'your_data.csv' # <--- 修改这里
    smiles_column = 'SMILES'
    target_column = 'ACTIVITY'
    save_dir = 'mindspore_mrsa_model'

    # --- 数据集划分参数 ---
    split_type = 'scaffold' # 可选 'random' 或 'scaffold'
    split_sizes = [0.8, 0.1, 0.1]

    # --- 模型超参数 ---
    hidden_size = 300
    depth = 3
    dropout = 0.1

    # --- 训练超参数 ---
    epochs = 30 # 可根据需要增加
    batch_size = 32
    learning_rate = 1e-4

args = Args()

## 3. 执行主流程

现在，我们按顺序执行数据加载、划分、训练和测试。

In [ ]:
# --- 1. 加载数据 ---
try:
    df = pd.read_csv(args.data_path)
    print(f"Successfully loaded data from '{args.data_path}'. Total molecules: {len(df)}")
    display(df.head())
except FileNotFoundError:
    print(f"Error: Data file not found at '{args.data_path}'")
    print("Please create a CSV file with 'SMILES' and 'ACTIVITY' columns, and update the `data_path` argument.")

if 'df' in locals():
    # --- 2. 划分数据集 ---
    train_data, val_data, test_data = split_data(
        df=df, 
        smiles_column=args.smiles_column, 
        target_column=args.target_column, 
        split_type=args.split_type, 
        split_sizes=args.split_sizes
    )

    # --- 3. 训练并获取最佳模型路径 ---
    best_model_path = run_training(args, train_data, val_data)

    # --- 4. 使用最佳模型进行测试 ---
    run_testing(args, test_data, best_model_path)